### Import

In [13]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import matplotlib.pyplot as plt

import copy
from typing import Sequence, Callable
from pennylane.tape import QuantumTape
# from pennylane import transform

A_to_au_conversion = 1.8897259885789

active_electrons = 4
active_orbitals = 4

### Structure

In [14]:
symbols = ["N", "H", "H", "N", "H", "H"]
# ground state coordinate: 
geometry = np.array([0.0, 0.0, 0.0 , 0.0, 0.0, 1.015264, 0.978541, 0.0, -0.270591, -0.627449, 1.276052, -0.477492 , -0.897827, 1.825923, 0.332013 , 0.080714, 1.825923, -0.953842])*A_to_au_conversion

### Project Setup

##### Generate qubit hamiltonian of the molecule

In [15]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=active_electrons,
    active_orbitals=active_orbitals,
    # method="pyscf"
)

In [16]:
H

<Hamiltonian: terms=313, wires=[0, 1, 2, 3, 4, 5, 6, 7]>


In [17]:
print(H)

  (-108.2509117996428) [I0]
+ (-0.22725040055629014) [Z7]
+ (-0.22725040055629003) [Z6]
+ (-0.14333511784309017) [Z5]
+ (-0.14333511784309005) [Z4]
+ (0.0635910339418741) [Z3]
+ (0.06359103394187421) [Z2]
+ (0.12111503176024321) [Z0]
+ (0.12111503176024331) [Z1]
+ (-3.2142223089043954e-08) [Y1 Y3]
+ (-3.2142223089043954e-08) [X1 X3]
+ (2.8583487677736662e-08) [Y0 Y2]
+ (2.8583487677736662e-08) [X0 X2]
+ (3.058345114789404e-08) [Y5 Y7]
+ (3.058345114789404e-08) [X5 X7]
+ (5.4224294522282716e-08) [Y4 Y6]
+ (5.4224294522282716e-08) [X4 X6]
+ (0.00031259622325041354) [Y2 Y4]
+ (0.00031259622325041354) [X2 X4]
+ (0.0038017750347166136) [Y3 Y5]
+ (0.0038017750347166136) [X3 X5]
+ (0.08346186127902747) [Z0 Z2]
+ (0.08346186127902747) [Z1 Z3]
+ (0.09129117225478453) [Z2 Z6]
+ (0.09129117225478453) [Z3 Z7]
+ (0.09299736444454987) [Z0 Z6]
+ (0.09299736444454987) [Z1 Z7]
+ (0.09617293025640639) [Z2 Z7]
+ (0.09617293025640639) [Z3 Z6]
+ (0.098188560548433) [Z0 Z7]
+ (0.098188560548433) [Z1 Z6]
+ (

In [18]:
print(qubits)

8


In [19]:
H.num_params

313

In [20]:
H.ops

[Identity(wires=[0]),
 PauliZ(wires=[0]),
 PauliY(wires=[0]) @ PauliZ(wires=[1]) @ PauliY(wires=[2]),
 PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliX(wires=[2]),
 PauliY(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliY(wires=[4]),
 PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliX(wires=[4]),
 PauliY(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]) @ PauliZ(wires=[5]) @ PauliY(wires=[6]),
 PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]) @ PauliZ(wires=[5]) @ PauliX(wires=[6]),
 PauliZ(wires=[2]),
 PauliZ(wires=[0]) @ PauliZ(wires=[2]),
 PauliY(wires=[2]) @ PauliZ(wires=[3]) @ PauliY(wires=[4]),
 PauliX(wires=[2]) @ PauliZ(wires=[3]) @ PauliX(wires=[4]),
 PauliZ(wires=[0]) @ PauliY(wires=[2]) @ PauliZ(wires=[3]) @ PauliY(wires=[4]),
 PauliZ(wires=[0]) @ PauliX(wires=[2]) @ PauliZ(wires=[3]) @ PauliX(wires=[4]),
 PauliY(wires=[2])

In [21]:
H.sparse_matrix

<bound method Hamiltonian.sparse_matrix of <Hamiltonian: terms=313, wires=[0, 1, 2, 3, 4, 5, 6, 7]>>

In [22]:
H.terms()

([tensor(-108.2509118, requires_grad=False),
  tensor(0.12111503, requires_grad=False),
  tensor(-4.53419948e-08, requires_grad=False),
  tensor(-4.53419948e-08, requires_grad=False),
  tensor(1.53796443e-08, requires_grad=False),
  tensor(1.53796443e-08, requires_grad=False),
  tensor(-0.01049284, requires_grad=False),
  tensor(-0.01049284, requires_grad=False),
  tensor(0.06359103, requires_grad=False),
  tensor(0.08346186, requires_grad=False),
  tensor(-0.01965167, requires_grad=False),
  tensor(-0.01965167, requires_grad=False),
  tensor(-0.01329028, requires_grad=False),
  tensor(-0.01329028, requires_grad=False),
  tensor(1.23018275e-08, requires_grad=False),
  tensor(1.23018275e-08, requires_grad=False),
  tensor(1.14747355e-08, requires_grad=False),
  tensor(1.14747355e-08, requires_grad=False),
  tensor(-0.14333512, requires_grad=False),
  tensor(0.11441429, requires_grad=False),
  tensor(4.78486417e-08, requires_grad=False),
  tensor(4.78486417e-08, requires_grad=False),
  t

In [23]:
print(qml.draw(H, decimals=None))

<function draw.<locals>.wrapper at 0x7f79e53c6480>


H

##### Generate Excitations

In [24]:
singles, doubles = qchem.excitations(active_electrons, qubits)

In [25]:
singles

[[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]

In [26]:
doubles

[[0, 1, 4, 5],
 [0, 1, 4, 7],
 [0, 1, 5, 6],
 [0, 1, 6, 7],
 [0, 2, 4, 6],
 [0, 3, 4, 5],
 [0, 3, 4, 7],
 [0, 3, 5, 6],
 [0, 3, 6, 7],
 [1, 2, 4, 5],
 [1, 2, 4, 7],
 [1, 2, 5, 6],
 [1, 2, 6, 7],
 [1, 3, 5, 7],
 [2, 3, 4, 5],
 [2, 3, 4, 7],
 [2, 3, 5, 6],
 [2, 3, 6, 7]]

In [27]:
print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 26


In [28]:
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
singles_excitations

[SingleExcitation(0.0, wires=[0, 4]),
 SingleExcitation(0.0, wires=[0, 6]),
 SingleExcitation(0.0, wires=[1, 5]),
 SingleExcitation(0.0, wires=[1, 7]),
 SingleExcitation(0.0, wires=[2, 4]),
 SingleExcitation(0.0, wires=[2, 6]),
 SingleExcitation(0.0, wires=[3, 5]),
 SingleExcitation(0.0, wires=[3, 7])]

In [29]:
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
doubles_excitations

[DoubleExcitation(0.0, wires=[0, 1, 4, 5]),
 DoubleExcitation(0.0, wires=[0, 1, 4, 7]),
 DoubleExcitation(0.0, wires=[0, 1, 5, 6]),
 DoubleExcitation(0.0, wires=[0, 1, 6, 7]),
 DoubleExcitation(0.0, wires=[0, 2, 4, 6]),
 DoubleExcitation(0.0, wires=[0, 3, 4, 5]),
 DoubleExcitation(0.0, wires=[0, 3, 4, 7]),
 DoubleExcitation(0.0, wires=[0, 3, 5, 6]),
 DoubleExcitation(0.0, wires=[0, 3, 6, 7]),
 DoubleExcitation(0.0, wires=[1, 2, 4, 5]),
 DoubleExcitation(0.0, wires=[1, 2, 4, 7]),
 DoubleExcitation(0.0, wires=[1, 2, 5, 6]),
 DoubleExcitation(0.0, wires=[1, 2, 6, 7]),
 DoubleExcitation(0.0, wires=[1, 3, 5, 7]),
 DoubleExcitation(0.0, wires=[2, 3, 4, 5]),
 DoubleExcitation(0.0, wires=[2, 3, 4, 7]),
 DoubleExcitation(0.0, wires=[2, 3, 5, 6]),
 DoubleExcitation(0.0, wires=[2, 3, 6, 7])]

In [30]:
operator_pool = doubles_excitations + singles_excitations
operator_pool

[DoubleExcitation(0.0, wires=[0, 1, 4, 5]),
 DoubleExcitation(0.0, wires=[0, 1, 4, 7]),
 DoubleExcitation(0.0, wires=[0, 1, 5, 6]),
 DoubleExcitation(0.0, wires=[0, 1, 6, 7]),
 DoubleExcitation(0.0, wires=[0, 2, 4, 6]),
 DoubleExcitation(0.0, wires=[0, 3, 4, 5]),
 DoubleExcitation(0.0, wires=[0, 3, 4, 7]),
 DoubleExcitation(0.0, wires=[0, 3, 5, 6]),
 DoubleExcitation(0.0, wires=[0, 3, 6, 7]),
 DoubleExcitation(0.0, wires=[1, 2, 4, 5]),
 DoubleExcitation(0.0, wires=[1, 2, 4, 7]),
 DoubleExcitation(0.0, wires=[1, 2, 5, 6]),
 DoubleExcitation(0.0, wires=[1, 2, 6, 7]),
 DoubleExcitation(0.0, wires=[1, 3, 5, 7]),
 DoubleExcitation(0.0, wires=[2, 3, 4, 5]),
 DoubleExcitation(0.0, wires=[2, 3, 4, 7]),
 DoubleExcitation(0.0, wires=[2, 3, 5, 6]),
 DoubleExcitation(0.0, wires=[2, 3, 6, 7]),
 SingleExcitation(0.0, wires=[0, 4]),
 SingleExcitation(0.0, wires=[0, 6]),
 SingleExcitation(0.0, wires=[1, 5]),
 SingleExcitation(0.0, wires=[1, 7]),
 SingleExcitation(0.0, wires=[2, 4]),
 SingleExcitation(

In [31]:
hf_state = qchem.hf_state(active_electrons, qubits)
hf_state

array([1, 1, 1, 1, 0, 0, 0, 0])

### Adaptive Optimizer

##### Helper Functions

In [32]:
from typing import Sequence, Callable
from pennylane import transforms

@transforms
def append_gate(tape: QuantumTape, params, gates) -> (Sequence[QuantumTape], Callable):
    """Append parameterized gates to an existing tape.

    Args:

        tape (QuantumTape or QNode or Callable): quantum circuit to transform by adding gates
        params (array[float]): parameters of the gates to be added
        gates (list[Operator]): list of the gates to be added

    Returns:
        qnode (QNode) or quantum function (Callable) or tuple[List[QuantumTape], function]: The transformed circuit as described in :func:`qml.transform <pennylane.transform>`.

    """
    new_operations = []

    for i, g in enumerate(gates):
        g = copy.copy(g)
        new_params = (params[i], *g.data[1:])
        g.data = new_params
        new_operations.append(g)

    new_tape = type(tape)(tape.operations + new_operations, tape.measurements, shots=tape.shots)

    def null_postprocessing(results):
        """A postprocesing function returned by a transform that only converts the batch of results
        into a result for a single ``QuantumTape``.
        """
        return results[0]  # pragma: no cover

    return [new_tape], null_postprocessing



TypeError: 'module' object is not callable

In [ ]:
from pennylane.optimize.adaptive import AdaptiveOptimizer

##### Adaptive Optimizer Class

In [ ]:
qml.optimize.AdaptiveOptimizer()

In [ ]:
class AdaptiveOptimizer:
    def __init__(self, param_steps=10, stepsize=0.5):
        self.param_steps = param_steps
        self.stepsize = stepsize
    
    @staticmethod
    def _circuit(params, gates, initial_circuit):
        final_circuit = append_gate(initial_circuit, params, gates)
        return final_circuit()
    
    def step(self, circuit, operator_pool, params_zero=True):
        return self.step_and_cost(circuit, operator_pool, params_zero=params_zero)
    
    def step_and_cost(self, circuit, operator_pool, drain_pool = False, params_zero=True):
        cost = circuit()
        qnode = copy.copy(circuit)

        if drain_pool:
            operator_pool = [
                gate
                for gate in operator_pool
                if all(
                    gate.name != operation.name or gate.wires != operation.wires
                    for operation in circuit.tape.operations
                )
            ]
        
        params = np.array([gate.parameters[0] for gate in operator_pool])
        qnode.func = self._circuit
        grads = qml.grad(qnode)(params, gates=operator_pool, initial_circuit=circuit.func)

        selected_gates = [operator_pool[np.argmax(abs(grads))]]
        optimizer = qml.GradientDescentOptimizer(stepsize=self.stepsize)

        if params_zero:
            params = np.zeros(len(selected_gates))
        else:
            params = np.array([gate.parameters[0] for gate in selected_gates], requires_grad = True)
        
        for _ in range(self.param_steps):
            params, _ = optimizer.step_and_cost(
                qnode, params, gates=selected_gates, initial_circuit=circuit.func
            )
        
        qnode.func = append_gate(circuit.func, params, selected_gates)

        return qnode, cost, max(abs(qml.math.toarray(grads)))

In [ ]:
from pennylane.optimize.adaptive import append_gate



### Custom Optimizer

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import copy

from pennylane import qchem
from pennylane import numpy as pnp
from pennylane.optimize.adaptive import AdaptiveOptimizer
from pennylane.optimize.adaptive import append_gate

A_to_au_conversion = 1.8897259885789

class CustomOptimizer(AdaptiveOptimizer):
    def custom_step(self, circuit, operator_pool, drain_pool=False, params_zero=True):
        cost = circuit()
        qnode = copy.copy(circuit)
        if drain_pool:
            operator_pool = [
                gate
                for gate in operator_pool
                if all(
                    gate.name != operation.name or gate.wires != operation.wires
                    for operation in circuit.tape.operations
                )
            ]

        params = pnp.array([gate.parameters[0] for gate in operator_pool], requires_grad=True)
        qnode.func = self._circuit
        grads = qml.grad(qnode)(params, gates=operator_pool, initial_circuit=circuit.func)
        selected_gates = [operator_pool[pnp.argmax(abs(grads))]]

        optimizer = qml.AdagradOptimizer(stepsize=self.stepsize)
        # === AdagradOptimizer, RMSPropOptimizer
        # === GradientDescentOptimizer, AdamOptimizer, MomentumOptimizer, NesterovMomentumOptimizer, QNGOptimizer, ShotAdaptiveOptimizer
        # === QNSPSAOptimizer, RiemannianGradientOptimizer, RotoselectOptimizer, RotosolveOptimizer, SPSAOptimizer

        if params_zero:
            params = pnp.zeros(len(selected_gates))
        else:
            params = pnp.array([gate.parameters[0] for gate in selected_gates], requires_grad=True)

        for _ in range(self.param_steps):
            params, _ = optimizer.step_and_cost(
                qnode, params, gates=selected_gates, initial_circuit=circuit.func
            )

        qnode.func = append_gate(circuit.func, params, selected_gates)

        return qnode, cost, max(abs(qml.math.toarray(grads)))


def calc_adapt_vqe_N2H4(threshold, active_electrons=4, active_orbitals=4):
    symbols = ["N", "H", "H", "N", "H", "H"]
    # ground state coordinate: 
    geometry = pnp.array([0.0, 0.0, 0.0 , 0.0, 0.0, 1.015264, 0.978541, 0.0, -0.270591, -0.627449, 1.276052, -0.477492 , -0.897827, 1.825923, 0.332013 , 0.080714, 1.825923, -0.953842])*A_to_au_conversion

    H, qubits = qchem.molecular_hamiltonian(
        symbols,
        geometry,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals,
        basis='sto-3g',
        # method="pyscf"
    )
    active_electrons = active_electrons
    singles, doubles = qchem.excitations(active_electrons, qubits)
    print(f"Total number of excitations = {len(singles) + len(doubles)}")
    singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
    doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
    operator_pool = doubles_excitations + singles_excitations   
    hf_state = qchem.hf_state(active_electrons, qubits)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit():
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        return qml.expval(H)
    energy_array = []

    
    opt = CustomOptimizer()
    for i in range(len(operator_pool)):
        circuit, energy, gradient = opt.custom_step(circuit, operator_pool)
        energy_array.append(energy)
        if i % 1 == 0:
            print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
            # print(qml.draw(circuit, decimals=None)())
            print()
        if gradient < threshold*10^(-threshold):
            break
    return energy_array, circuit

config = [[2,2]]
threshold = 3
setting='test'

for i in range(len(config)):
    print("Configuration: ", threshold, config[i][0], config[i][1])
    E, circuit = calc_adapt_vqe_N2H4(threshold, active_electrons=config[i][0], active_orbitals=config[i][1])
    
    file_path = f"data/N2H4_{threshold}_{config[i][0]}_{config[i][1]}_{setting}.txt"
    # Open the file in write mode
    with open(file_path, "w") as file:
        # Write each element of the array to a new line
        for element in E:
            file.write(str(element) + "\n")

Configuration:  3 2 2
Total number of excitations = 3
n = 0,  E = -109.73067811 H, Largest Gradient = 0.031

n = 1,  E = -109.73176676 H, Largest Gradient = 0.000

n = 2,  E = -109.73176739 H, Largest Gradient = 0.000



In [ ]:
circuit

<QNode: device='<default.qubit device (wires=4) at 0x7fdcca14d910>', interface='auto', diff_method='best'>

In [ ]:
print(qml.draw(circuit, decimals=None)())

0: ──X─╭G²────╭G─┤ ╭<𝓗>
1: ──X─├G²─╭G─│──┤ ├<𝓗>
2: ────├G²─│──╰G─┤ ├<𝓗>
3: ────╰G²─╰G────┤ ╰<𝓗>


In [ ]:
circuit.qtape

<QuantumScript: wires=[0, 1, 2, 3], params=3>